In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.cm
import numpy as np
import urllib.parse
import requests
import simplejson as json
%matplotlib inline
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

In [12]:
data_df = pd.read_csv("Existing_Commercial_Buildings_Energy_Performance_Ordinance_Report.csv")

In [14]:
data_df['Full.Address'][16]

'1001 COLUMBUS AVE\nSAN FRANCISCO, CA 94133\n(37.803867, -122.415419)'

In [15]:
data_df.columns
data_df['Building Address']
data_df = data_df.fillna('0')
data_df['lat'] = ""
data_df['long'] = ""
data_df.head()

,Parcel(s),Building Name,Building Address,Postal Code,Full.Address,Floor Area,Property Type - Self Selected,PIM Link,Benchmark 2017 Status,2017 Reason for Exemption,...,2011 Site EUI (kBtu/ft2),2011 Source EUI (kBtu/ft2),2011 Percent Better than National Median Site EUI,2011 Percent Better than National Median Source EUI,2011 Total GHG Emissions (Metric Tons CO2e),2011 Total GHG Emissions Intensity (kgCO2e/ft2),2011 Weather Normalized Site EUI (kBtu/ft2),2011 Weather Normalized Source EUI (kBtu/ft2),lat,long
0,5584/001B,Apparel Triangle,20 APPAREL WAY,94124,0,"34,537",Non-Refrigerated Warehouse,0,Violation - Did Not Report,0,...,0,0,0,0,0,0,0,0,,
1,3957/002,350 Rhode Island - North,350 Rhode Island - North,94103,"350 Rhode Island North\nSAN FRANCISCO, CA 94103\n","127,074",Other - Education,http://propertymap.sfplanning.org/?&search=395...,Complied,0,...,125,0,96,0,"1,160.87",0,123.6,0,,
2,5584/001A,Apparel Triangle,425-495 Barneveld Avenue,94124,0,"34,537",Non-Refrigerated Warehouse,0,Violation - Did Not Report,0,...,0,0,0,0,0,0,0,0,,
3,5587/004A,2222 PALOU AVE,2222 PALOU AVE,94124,0,"44,311",Non-Refrigerated Warehouse,0,Violation - Did Not Report,0,...,0,0,0,0,0,0,0,0,,
4,5587/004B,2160-2196 Palou Ave,2160 Palou Ave,94124,0,"44,311",0,0,Violation - Did Not Report,0,...,0,0,0,0,0,0,0,0,,


In [16]:
def geocode(address):
    params = { 'format'        :'json', 
               'addressdetails': 1, 
               'q'             : address}
    return requests.get('http://nominatim.openstreetmap.org/search', params=params)


k = range(len(data_df))
for address,building_address,i,pin in zip(data_df['Full.Address'],data_df['Building Address'],k,data_df['Postal Code']):
    if len(address.split("(")) >1:
        temp = address.split("(")[1]
        lati,longi = temp.split(",")
        longi = longi[:-1]
        #print(lati,longi)
        data_df.loc[i,'lat'] = lati
        data_df.loc[i,'long'] = longi
    else:
        #if address == '0':
        address = building_address
        address += ", San Francisco,"+str(pin)
        #print(address)
        response = geocode(address)
        if len(response.json()) != 0:
            #continue
            #print("----")
            data_df.loc[i,'lat'] = response.json()[0]['lat']
            data_df.loc[i,'long'] = response.json()[0]['lon']
            

In [17]:
data_df.to_csv("energy_with_location.csv")

In [95]:
df = pd.read_csv("energy_with_location.csv")
df.head()

,Unnamed: 0,Parcel(s),Building Name,Building Address,Postal Code,Full.Address,Floor Area,Property Type - Self Selected,PIM Link,Benchmark 2017 Status,...,2011 Site EUI (kBtu/ft2),2011 Source EUI (kBtu/ft2),2011 Percent Better than National Median Site EUI,2011 Percent Better than National Median Source EUI,2011 Total GHG Emissions (Metric Tons CO2e),2011 Total GHG Emissions Intensity (kgCO2e/ft2),2011 Weather Normalized Site EUI (kBtu/ft2),2011 Weather Normalized Source EUI (kBtu/ft2),lat,long
0,0,5584/001B,Apparel Triangle,20 APPAREL WAY,94124,0,"34,537",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.740240,-122.402721
1,1,3957/002,350 Rhode Island - North,350 Rhode Island - North,94103,"350 Rhode Island North\nSAN FRANCISCO, CA 94103\n","127,074",Other - Education,http://propertymap.sfplanning.org/?&search=395...,Complied,...,125,0.0,96,0.0,"1,160.87",0.0,123.6,0,37.765388,-122.403103
2,2,5584/001A,Apparel Triangle,425-495 Barneveld Avenue,94124,0,"34,537",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,NaN,NaN
3,3,5587/004A,2222 PALOU AVE,2222 PALOU AVE,94124,0,"44,311",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.740549,-122.401790
4,4,5587/004B,2160-2196 Palou Ave,2160 Palou Ave,94124,0,"44,311",0,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.739940,-122.401016


In [19]:
df.dropna(subset=['lat'])

,Unnamed: 0,Parcel(s),Building Name,Building Address,Postal Code,Full.Address,Floor Area,Property Type - Self Selected,PIM Link,Benchmark 2017 Status,...,2011 Site EUI (kBtu/ft2),2011 Source EUI (kBtu/ft2),2011 Percent Better than National Median Site EUI,2011 Percent Better than National Median Source EUI,2011 Total GHG Emissions (Metric Tons CO2e),2011 Total GHG Emissions Intensity (kgCO2e/ft2),2011 Weather Normalized Site EUI (kBtu/ft2),2011 Weather Normalized Source EUI (kBtu/ft2),lat,long
0,0,5584/001B,Apparel Triangle,20 APPAREL WAY,94124,0,"34,537",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.740240,-122.402721
1,1,3957/002,350 Rhode Island - North,350 Rhode Island - North,94103,"350 Rhode Island North\nSAN FRANCISCO, CA 94103\n","127,074",Other - Education,http://propertymap.sfplanning.org/?&search=395...,Complied,...,125,0.0,96,0.0,"1,160.87",0.0,123.6,0,37.765388,-122.403103
3,3,5587/004A,2222 PALOU AVE,2222 PALOU AVE,94124,0,"44,311",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.740549,-122.401790
4,4,5587/004B,2160-2196 Palou Ave,2160 Palou Ave,94124,0,"44,311",0,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.739940,-122.401016
5,5,3741/035,2 Folsom,250 THE EMBARCADERO,94105,"250 THE\nSAN FRANCISCO, CA 94105\n","700,079",Office,http://propertymap.sfplanning.org/?&search=374...,Complied,...,58.5,0.0,-33.8,0.0,"2,716.55",0.0,55.9,0,37.791775,-122.390360
6,6,1691/019,850 La Playa,850 LA PLAYA,94121,"850 LA\nSAN FRANCISCO, CA 94121\n","97,422",Supermarket/Grocery Store,http://propertymap.sfplanning.org/?&search=169...,Violation - Data Not Verified,...,276,0.0,20.4,0.0,810.77,0.0,276,0,37.772422,-122.509362
7,7,5586/001A,Barneveld Dorman 65-81,65-81 Dorman Ave,94124,0,"37,550",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.739483,-122.403247
8,8,5586/001B,Barneveld Dorman 1-55,1-55 Dorman Ave,94124,0,"37,550",Non-Refrigerated Warehouse,0,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.739991,-122.401432
11,11,2116/028,1900 19TH AVE,1900 19TH AVE,94116,"1900 19TH AVE\nSAN FRANCISCO, CA 94116\n(37.75...","23,872",0,http://propertymap.sfplanning.org/?&search=211...,Violation - Did Not Report,...,0,0.0,0,0.0,0,0.0,0,0,37.752088,-122.476213
12,12,3715/001,1 MISSION,100 The Embarcadero,94105,"100 The\nSAN FRANCISCO, CA 94105\n","31,740",Office,http://propertymap.sfplanning.org/?&search=371...,Complied,...,0,0.0,0,0.0,0,0.0,0,0,37.793322,-122.392775


In [20]:
new_df = df.dropna(subset=['lat'])
len(new_df)


1631

In [ ]:

def replaceComma(pos):
    return pos.replace(",","")
new_df['2016 Site EUI (kBtu/ft2)']=new_df['2016 Site EUI (kBtu/ft2)'].apply(replaceComma)
new_df['Floor Area']=new_df['Floor Area'].apply(replaceComma)
new_df['Floor Area'] = pd.to_numeric(new_df['Floor Area'])
new_df['2016 Site EUI (kBtu/ft2)']=pd.to_numeric(new_df['2016 Site EUI (kBtu/ft2)'])
#new_df.head()

def Normalize(feature_name,df):
    max_value = df[feature_name].max()
    min_value = df[feature_name].min()
    return (df[feature_name] - min_value) / (max_value - min_value)


new_df['Normalized_FloorArea'] = Normalize('Floor Area',new_df)
new_df['Normalized_2016EUI'] = Normalize('2016 Site EUI (kBtu/ft2)',new_df)

In [ ]:
figf , ax = plt.subplots(figsize=(10,20))
m = Basemap(resolution='h', # c, l, i, h, f or None
            projection='merc',
            lat_0=37.76, lon_0=-122.42,
            llcrnrlon=-122.56473, llcrnrlat=37.692998, urcrnrlon=-122.34178, urcrnrlat=37.833429)
m.drawmapboundary(fill_color='#46bcec')
#m.drawcountries()
#m.drawstates()
m.fillcontinents(color='#f2f2f2',zorder=0)
m.drawcoastlines()
    
#new_df['Building Name'].apply(plot_area)    
for lat, long,area,cons  in zip(new_df['lat'],new_df['long'],new_df['Normalized_FloorArea'] ,new_df['2016 Site EUI (kBtu/ft2)']):     #print(lat,long)
    x, y = m(long,lat)
    size = area *120
    cons = cons *10
    #print(cons)
    #m.plot(x,y, 'o',markersize = size, color ='#444444', alpha=0.8)
    m.scatter(x,y, label=None,c=cons, cmap='viridis',s=size, alpha=0.8)

m.readshapefile('SF_Centrelines/geo_export_acea975e-e38d-45e2-a310-1bdb64c62d52', 'areas')
#     #break

# lat, lon = new_df['lat'], new_df['long']
# population, area = new_df['2016 Site EUI (kBtu/ft2)'], new_df['Floor Area']

# Scatter the points, using size and color but no label
# m.scatter(lon, lat, label=None,
#             c=population, cmap='viridis',
#             s=area, linewidth=0, alpha=1)
plt.savefig("SF_2016_E.png")

In [ ]:
#plt.colorbar()
plt.show()

In [76]:
df_partial = new_df.head(20)
df_partial['2016 Site EUI (kBtu/ft2)']

0      21.2
1     100.4
3      23.0
4      11.4
5      53.6
6     265.1
7      30.4
8       6.6
11      0.0
12    236.3
16      0.0
17     52.3
19      0.0
20    224.3
21     38.6
22      0.0
23      0.0
24    136.3
26     25.5
27      0.0
Name: 2016 Site EUI (kBtu/ft2), dtype: float64

In [96]:
import folium
import branca.colormap as cm

SF_COORDINATES = (37.76, -122.45)
map_osm = folium.Map(location=SF_COORDINATES, zoom_start=13)
#map_osm = folium.Map(prefer_canvas=True)
color_var = '2016 Site EUI (kBtu/ft2)' #what variable will determine the color
cmap = cm.LinearColormap(['blue', 'red'],
                         vmin=new_df[color_var].quantile(0.05), vmax=new_df[color_var].quantile(0.95),
                         caption = color_var)

def plotDot(df):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[df['lat'], df['long']],
                        radius= df['Normalized_FloorArea'] * 25,
                        weight=3,#remove outline
                        #popup = df['Building Name'],
                        fill=True,
                        color=cmap(df[color_var]),
                        fillcolor=cmap(df[color_var])).add_to(map_osm)


#df_partial.apply(plotDot, axis = 1)  
new_df.apply(plotDot, axis = 1) 
map_osm.fit_bounds(map_osm.get_bounds())
map_osm
map_osm.save('map.html')